In [9]:
import cv2
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input , Lambda , GlobalAveragePooling2D , Dropout, Dense
from tensorflow.keras.callbacks import ModelCheckpoint

In [10]:
train_paths = ['C:\\Users\\saran\\Desktop\\web_app\\intel_image\\seg_train\\seg_train\\' + x for x in os.listdir('C:\\Users\\saran\\Desktop\\web_app\\intel_image\\seg_train\\seg_train')] 
test_paths = ['C:\\Users\\saran\\Desktop\\web_app\\intel_image\\seg_test\\seg_test\\' + x for x in os.listdir('C:\\Users\\saran\\Desktop\\web_app\\intel_image\\seg_test\\seg_test')] 

print('\ntrain images\n')
for path in train_paths:
    print(f'{path} -> {len(os.listdir(path))}')

print('\ntest images\n')
for path in test_paths:
    print(f'{path} -> {len(os.listdir(path))}')
    


train images

C:\Users\saran\Desktop\web_app\intel_image\seg_train\seg_train\buildings -> 2191
C:\Users\saran\Desktop\web_app\intel_image\seg_train\seg_train\forest -> 2271
C:\Users\saran\Desktop\web_app\intel_image\seg_train\seg_train\glacier -> 2404
C:\Users\saran\Desktop\web_app\intel_image\seg_train\seg_train\mountain -> 2512
C:\Users\saran\Desktop\web_app\intel_image\seg_train\seg_train\sea -> 2274
C:\Users\saran\Desktop\web_app\intel_image\seg_train\seg_train\street -> 2382

test images

C:\Users\saran\Desktop\web_app\intel_image\seg_test\seg_test\buildings -> 437
C:\Users\saran\Desktop\web_app\intel_image\seg_test\seg_test\forest -> 474
C:\Users\saran\Desktop\web_app\intel_image\seg_test\seg_test\glacier -> 553
C:\Users\saran\Desktop\web_app\intel_image\seg_test\seg_test\mountain -> 525
C:\Users\saran\Desktop\web_app\intel_image\seg_test\seg_test\sea -> 510
C:\Users\saran\Desktop\web_app\intel_image\seg_test\seg_test\street -> 501


In [11]:
train_dir = 'C:\\Users\\saran\\Desktop\\web_app\\intel_image\\seg_train\\seg_train\\'
test_dir = 'C:\\Users\\saran\\Desktop\\web_app\\intel_image\\seg_test\\seg_test\\'

In [12]:
train_data_gen = ImageDataGenerator(horizontal_flip=True).flow_from_directory(train_dir , target_size=(150,150) , color_mode='rgb' , batch_size=32, class_mode='categorical' , shuffle=True)
test_data_gen = ImageDataGenerator().flow_from_directory(test_dir , target_size=(150,150) , color_mode='rgb' , batch_size=32, class_mode='categorical' , shuffle=True)

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [13]:
class_mapping = dict((v,k) for k,v in train_data_gen.class_indices.items())
class_mapping

{0: 'buildings',
 1: 'forest',
 2: 'glacier',
 3: 'mountain',
 4: 'sea',
 5: 'street'}

In [14]:
input_layer = Sequential([Input((150,150,3)) , Lambda(preprocess_input)])
mobilenet = MobileNetV2(input_shape=(150,150,3) , include_top=False)
output_layer = Sequential([GlobalAveragePooling2D(),
                           Dropout(0.4),
                           Dense(6 , activation=('softmax'))])

model = Sequential([input_layer , mobilenet , output_layer])

In [15]:
model.compile(optimizer='adam' , loss='categorical_crossentropy' , metrics=['accuracy'])
model.build(((None,150,150,3)))
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_3 (Sequential)   (None, 150, 150, 3)       0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 5, 5, 1280)       2257984   
 ional)                                                          
                                                                 
 sequential_4 (Sequential)   (None, 6)                 7686      
                                                                 
Total params: 2,265,670
Trainable params: 2,231,558
Non-trainable params: 34,112
_________________________________________________________________


In [19]:
call_back = ModelCheckpoint('model/' , save_best_only=True)
model.fit(train_data_gen, validation_data=test_data_gen , callbacks=[call_back] ,epochs=20)

Epoch 1/20
186/439 [===========>..................] - ETA: 5:12 - loss: 0.3551 - accuracy: 0.8825

KeyboardInterrupt: 

In [30]:
import tensorflow as tf
tflite_model = tf.lite.TFLiteConverter.from_keras_model(model).convert()
with open('model.tflite' , 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\saran\AppData\Local\Temp\tmp8xaf3ene\assets


INFO:tensorflow:Assets written to: C:\Users\saran\AppData\Local\Temp\tmp8xaf3ene\assets
